In [ ]:
# This notebook demonstrates how to calculate 3D Zernike moments using ZMPY3D with PyTorch.
#
# This notebook primarily consists of the following steps:
#     1. Install ZMPY3D_PT.
#     2. Define necessary parameters.
#     3. Load precalculated cache.
#     4. Download example PDB data with coordinates.
#     5. Convert coordinate data into a voxel.
#     6. Create a callable function for generating Zernike moments and normalization.
#     7. Obtain the results.
#     8. A command line interface (CLI) example

In [ ]:
# Install ZMPY3D versions for PyTorch.
! pip install ZMPY3D-PT
print(f"It is recommended to restart the Python kernel for the IPython notebook.")


In [ ]:
# Download example data from GitHub using curl
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/1WAC_A.txt


In [ ]:
import ZMPY3D_PT as z
import torch
import pickle

if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("CUDA is available, PyTorch uses GPU.")
else:
    print("CUDA is unavailable. PyTorch uses CPU.")

MaxOrder = 20 # Set the maximum order to 20
GridWidth= 1.00 # Set the grid width to 1.00 (equivalent to 1 angstrom)
Param=z.get_global_parameter() # Retrieve the necessary parameters

# Find the cache_data directory based on the site package location of ZMPY3D_PT.
LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

# Convert the precalculated cache data into PyTorch tensor objects
GCache_pqr_linear = torch.tensor(CachePKL['GCache_pqr_linear'])
GCache_complex = torch.tensor(CachePKL['GCache_complex'])
GCache_complex_index = torch.tensor(CachePKL['GCache_complex_index'])
CLMCache3D = torch.tensor(CachePKL['CLMCache3D'], dtype=torch.complex128)
CLMCache = torch.tensor(CachePKL['CLMCache'], dtype=torch.float64)

print(f"Now using the MaxOrder of {MaxOrder} and the GridWidth of {GridWidth}.")
print(f"Pre-calculated parameters have been loaded successfully.")


In [ ]:
%%time
PDBFileName='./1WAC_A.txt'

# Convert structure data into coordinates
[XYZ,AA_NameList]=z.get_pdb_xyz_ca(PDBFileName)

# Convert coordinates into voxels using precalculated Gaussian densities
ResidueBox=z.get_residue_gaussian_density_cache(Param)
[Voxel3D,Corner]=z.fill_voxel_by_weight_density(XYZ,AA_NameList,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])

# Convert the voxel data into a tensor object
Voxel3D=torch.tensor(Voxel3D,dtype=torch.float64)

print(f"Converting PDB to 3D voxel grid with NumPy on CPU, then transferring to GPU memory as PyTorch objects.")
print(f"Time elapsed is as follows:")

In [ ]:
%%time

Dimension_BBox_scaled=Voxel3D.shape
MaxOrder=torch.tensor(MaxOrder,dtype=torch.int64)

X_sample = torch.arange(0, Dimension_BBox_scaled[0] + 1, dtype=torch.float64)
Y_sample = torch.arange(0, Dimension_BBox_scaled[1] + 1, dtype=torch.float64)
Z_sample = torch.arange(0, Dimension_BBox_scaled[2] + 1, dtype=torch.float64)

# Calculate the volume mass and the center of mass
[VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)

# Calculate the weights for sphere sampling
[AverageVoxelDist2Center,MaxVoxelDist2Center]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,torch.tensor(Param['default_radius_multiplier'], dtype=torch.float64))

Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)

# Apply weights to the geometric moments
_,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                  ,MaxOrder
                                  ,Sphere_X_sample
                                  ,Sphere_Y_sample
                                  ,Sphere_Z_sample)

# Convert to scaled 3D Zernike moments
ZMoment_scaled,ZMoment_raw=z.calculate_bbox_moment_2_zm(MaxOrder
                                    , GCache_complex
                                    , GCache_pqr_linear
                                    , GCache_complex_index
                                    , CLMCache3D
                                    , SphereBBoxMoment)

# Convert the scaled 3D Zernike moments into 3DZD-based descriptors
ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
ZM_3DZD_invariant_121=torch.flatten(ZM_3DZD_invariant[~torch.isnan(ZM_3DZD_invariant)])


print(f"The dimensions of the voxel being used are {Voxel3D.shape}.")
print(f"Transforming the 3D voxel into Zernike moments with global normalization, 3DZD style, yields 121 descriptors.")
print(f"Time elapsed is as follows:")


In [ ]:
def OneTimeConversion_PT(Voxel3D,MaxOrder):
    Dimension_BBox_scaled=Voxel3D.shape
    MaxOrder=torch.tensor(MaxOrder,dtype=torch.int64)

    X_sample = torch.arange(0, Dimension_BBox_scaled[0] + 1, dtype=torch.float64)
    Y_sample = torch.arange(0, Dimension_BBox_scaled[1] + 1, dtype=torch.float64)
    Z_sample = torch.arange(0, Dimension_BBox_scaled[2] + 1, dtype=torch.float64)

    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)

    [AverageVoxelDist2Center,_]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,torch.tensor(Param['default_radius_multiplier'], dtype=torch.float64))

    Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)

    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                      ,MaxOrder
                                      ,Sphere_X_sample
                                      ,Sphere_Y_sample
                                      ,Sphere_Z_sample)

    ZMoment_scaled,_=z.calculate_bbox_moment_2_zm(MaxOrder
                                        , GCache_complex
                                        , GCache_pqr_linear
                                        , GCache_complex_index
                                        , CLMCache3D
                                        , SphereBBoxMoment)

    ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
    ZM_3DZD_invariant_121=torch.flatten(ZM_3DZD_invariant[~torch.isnan(ZM_3DZD_invariant)])

    return ZM_3DZD_invariant_121

print(f"Merge all steps into a single callable PyTorch function, OneTimeConversion_PT.")




In [ ]:
%%time
# Use OneTimeConversion_PT to calculate and normalize 3D Zernike moments
print(OneTimeConversion_PT(Voxel3D,MaxOrder))

In [ ]:
# Alternatively, use a system call to compute results via CLI
# ./ZMPY3D_PT_CLI_ZM PDBFile GridWidth MaximumOrder NormOrder Mode
! ZMPY3D_PT_CLI_ZM "./1WAC_A.txt" 1.0 20 2 1
